In [4]:
import numpy as np
import random
from collections import defaultdict

class MonteCarlo:
    def __init__(self, env, gamma=0.9):
        """
        Initialize the Monte Carlo instance.
        :param env: OpenAI Gym-like environment.
        :param gamma: Discount factor.
        """
        self.env = env
        self.gamma = gamma
        self.returns = defaultdict(list)  # Store returns for each state-action pair
        self.state_values = defaultdict(float)  # For prediction
        self.action_values = defaultdict(float)  # For control
        self.policy = defaultdict(lambda: random.choice(range(env.action_space.n)))  # Random initial policy

    def generate_episode(self, policy):
        """
        Generate an episode following the policy.
        :param policy: Dictionary mapping state to action.
        :return: List of (state, action, reward) tuples.
        """
        episode = []
        state = self.env.reset()
        while True:
            action = policy[state]
            next_state, reward, done, _ = self.env.step(action)
            episode.append((state, action, reward))
            state = next_state
            if done:
                break
        return episode

    def mc_prediction(self, episodes=500):
        """
        Monte Carlo Prediction for estimating state-value function V(s).
        :param episodes: Number of episodes to sample.
        """
        for _ in range(episodes):
            episode = self.generate_episode(self.policy)
            G = 0
            visited_states = set()
            for t in reversed(range(len(episode))):
                state, _, reward = episode[t]
                G = self.gamma * G + reward
                if state not in visited_states:  # First-visit MC
                    visited_states.add(state)
                    self.returns[state].append(G)
                    self.state_values[state] = np.mean(self.returns[state])

    def mc_control(self, episodes=500):
        """
        Monte Carlo Control using Exploring Starts to find the optimal policy.
        :param episodes: Number of episodes to sample.
        """
        for _ in range(episodes):
            episode = self.generate_episode(self.policy)
            G = 0
            visited_state_actions = set()
            for t in reversed(range(len(episode))):
                state, action, reward = episode[t]
                G = self.gamma * G + reward
                if (state, action) not in visited_state_actions:  # First-visit MC
                    visited_state_actions.add((state, action))
                    self.returns[(state, action)].append(G)
                    self.action_values[(state, action)] = np.mean(self.returns[(state, action)])

                    # Update policy to be greedy w.r.t the action-value function
                    state_actions = [self.action_values[(state, a)] for a in range(self.env.action_space.n)]
                    self.policy[state] = np.argmax(state_actions)

**Inference**

Monte-carlo prediction

Goal: Learn how good it is to be in a certain state when following a specific policy.

Imagine playing a game (like Blackjack) over and over while following a certain strategy (policy).
Every time you reach a state (e.g., your current cards in Blackjack), you note down the eventual total reward you get by the end of the game.
After enough games, you take the average of all these total rewards for each state. *This average becomes your estimate of how valuable that state is (called V(s)) when using that strategy. Think of it as keeping track of "how good" each state is by observing what happens when you visit it many times.

Instead of just tracking how good each state is, you now track how good each action is in each state. This is called Q(s,a), the action-value function.
At the start, you try actions randomly (this is called exploration).
Over time, you see which actions lead to the best outcomes and start choosing those actions more often (this is called exploitation).
To keep improving, you mix random actions (exploration) and smart actions (exploitation) using a method called epsilon-greedy: most of the time, you pick the best-known action, but sometimes you try something random to learn more.
After many games, the strategy becomes optimized because it picks the best actions based on what you’ve learned.
Think of this as learning the best way to play the game by trial and error over thousands of games.

In [5]:
# Example Usage
if __name__ == "__main__":
    import gym
    env = gym.make("Taxi-v3")

    mc_agent = MonteCarlo(env)
    print("Initial Random Policy:")
    print(mc_agent.policy)

    print("\nPerforming MC Prediction...")
    mc_agent.mc_prediction(episodes=1000)
    print("Estimated State Values:")
    print(dict(mc_agent.state_values))

    print("\nPerforming MC Control...")
    mc_agent.mc_control(episodes=1000)
    print("Optimized Policy:")
    print(dict(mc_agent.policy))

/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


Initial Random Policy:
defaultdict(<function MonteCarlo.__init__.<locals>.<lambda> at 0x7bc6f7981a20>, {})

Performing MC Prediction...
Estimated State Values:
{306: -1.0, 427: -10.0, 224: -1.5727272727272728, 244: -1.3272727272727272, 124: -9.999999992357, 474: -10.0, 131: -2.442142857142857, 111: -2.461, 11: -1.8884285714285713, 31: -2.2574285714285716, 413: -10.0, 483: -10.0, 289: -10.0, 282: -10.0, 182: -90.99999992944923, 208: -10.0, 108: -90.99999992944923, 151: -10.0, 401: -1.0, 448: -1.0, 454: -1.4500000000000002, 434: -1.45, 351: -1.6299999999999997, 251: -1.27, 334: -10.0, 148: -10.0, 61: -10.0, 42: -1.0, 344: -10.0, 281: -1.1500000000000001, 381: -1.7499999999999998, 181: -9.999999992944923, 132: -1.0, 189: -90.99999992944923, 112: -9.999999992631365, 389: -1.09, 369: -1.8099999999999998, 171: -10.0, 191: -90.99999992944923, 363: -1.5142857142857142, 263: -1.3857142857142857, 491: -1.3857142857142857, 391: -1.5142857142857142, 447: -1.0, 284: -10.0, 264: -90.99999992944923, 

**Environment Details (Taxi-v3):**
State Space: 500 discrete states, representing the taxi’s position, the passenger’s location, and the destination.
Action Space: 6 discrete actions:
Move South (0), North (1), East (2), West (3), Pickup (4), Dropoff (5).
Goal: Minimize the number of moves to pick up and drop off the passenger correctly.